In [1]:
pip install pandas scikit-learn peft datasets tensorboardX numba bitsandbytes


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, \
                         TrainingArguments, Trainer, DataCollatorWithPadding, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, make_scorer
import os
import torch
import torch.nn as nn
import pdb

## Google drive mount

from google.colab import drive
drive.mount("/content/drive")

# Dataset uploading

In [3]:
DATASET_PATHS = {
    "local": {
        "train": "../../datasets/train_set.csv",
        "test": "../../datasets/test_set.csv"
    },
    "local_two": {
        "train": "train_set.csv",
        "test": "test_set.csv"
    },
    "local_three": {
        "train": "drive/MyDrive/fine_tuning/train_set.csv",
        "test": "drive/MyDrive/fine_tuning/test_set.csv"
    },

    "kaggle": {
        "train": "/kaggle/input/python-codes-time-complexity/train_set.csv",
        "test": "/kaggle/input/python-codes-time-complexity/test_set.csv"
    }
}

def upload_datasets(dataset_paths=DATASET_PATHS):
    for path in dataset_paths:
        if os.path.exists(dataset_paths[path]['train']) and os.path.exists(dataset_paths[path]['test']):
            return dataset_paths[path]['train'], dataset_paths[path]['test']

    return FileNotFoundError(f"Datasets do not exist in the current paths: {dataset_paths}")


train_set_path, test_set_path = upload_datasets()

# Metrics

### Ordering labels by Hierarchy

In [4]:
LABELS_HIERARCHY = {
    'constant': 1,
    'logn': 2,
    'linear': 3,
    'nlogn': 4,
    'quadratic': 5,
    'cubic': 6,
    'np': 7
}

N_CLASSES = len(LABELS_HIERARCHY)

# Dataset uploading

In [5]:
train_set = load_dataset("csv", data_files=train_set_path)['train']
test_set = load_dataset("csv", data_files=test_set_path)['train']

train_labels = train_set['complexity']
test_labels = test_set['complexity']

# Evaluating

### Writing the custom metric *Hierarchy Complexity Score*

In [6]:
def hc_score(y_true, y_pred, n_classes=N_CLASSES):
    assert len(y_true) == len(y_pred), f"The amount of y_true labels: {len(y_true)} does not equal to the amount of y_pred: {len(y_pred)}."

    n_samples = len(y_true)

    return (np.sum(np.abs(y_pred - y_true)) / n_classes) / n_samples

## Computing metrics

In [7]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    preds = np.argmax(logits[0], axis=-1) if isinstance(logits, tuple) else np.argmax(logits, axis=-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)
    # Calculate F-1 Macro
    f1_macro_score = f1_score(labels, preds, average='macro')
    # Calculate Hierarchy Score
    hierarchy_score = hc_score(labels, preds)

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro_score,
        "hierarchy_score": hierarchy_score
    }

# Tokenizing

## Label tokenizing

In [8]:
labelEncoder = LabelEncoder()
labelEncoder.fit(train_set['complexity'])

LabelEncoder()

## Feature tokenizing

In [9]:
def tokenize_data(samples, tokenizer):
    tokenized = tokenizer(samples['code'], truncation=True, max_length=512)
    tokenized['labels'] = labelEncoder.transform(samples['complexity'])
    return tokenized


def set_tokenizer(checkpoint):
    try:
        tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token = "<pad>")
    except Exception as e:
        print(f"Failed to load {checkpoint}: {e}")
        checkpoint = "-".join(checkpoint.split("-")[:2])
        tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        print(f"Falling back to {checkpoint}")

    X_train = train_set.map(lambda x: tokenize_data(x, tokenizer), batched=True)
    X_eval = test_set.map(lambda x: tokenize_data(x, tokenizer), batched=True)

    # Collator for batch padding
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    return tokenizer, data_collator, X_train, X_eval

# Model

## Device

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Checkpoint

In [11]:
checkpoint = "deepseek-ai/DeepSeek-Coder-V2-Lite-Base"

## Quantizing

In [12]:
# Configure 4-bit quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16

)

## Model loading

In [13]:
def set_model(checkpoint):
    model = AutoModel.from_pretrained(checkpoint, torch_dtype='bfloat16', num_labels=7,
                                      trust_remote_code=True, quantization_config=quant_config)
    # Configuring padding token in case is absent
    model.config.pad_token_id = tokenizer.pad_token_id
    # As well, as resizing the embeddings to accomodate the new *pad* token
    model.resize_token_embeddings(len(tokenizer))


    return model

## Classifier head

In [14]:
from transformers import AutoModelForCausalLM, AutoConfig, PreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput

class DeepseekV2ForSequenceClassification(PreTrainedModel):
    config_class = AutoConfig

    def __init__(self, base_model, config):
      super().__init__(config)
      self.num_labels = config.num_labels
      self.model = base_model

      self.dense = nn.Linear(config.hidden_size, config.num_labels, device=self.model.device, dtype=config.torch_dtype)
      # Initialize weights and apply final processing
      self.post_init()

    def forward(self, input_ids, attention_mask, labels=None, *args, **kwargs):
      outputs = self.model(input_ids, attention_mask)

      hidden_states = outputs.last_hidden_state
      logits = self.dense(hidden_states)

      # Batch size
      if input_ids is not None:
        batch_size = input_ids.shape[0]

      # If padding token id is not configured and the batch size is > 1
      if self.config.pad_token_id is None and batch_size != 1:
        raise ValueError("Cannot handle batch sizes > 1 if no padding token is defined.")
      # If padding token id is not configured
      if self.config.pad_token_id is None:
        last_non_pad_token = -1
      # if encoded inputs exist => find the last non padded token to pool data from
      elif input_ids is not None:
        non_pad_mask = (input_ids != self.config.pad_token_id).to(logits.device, dtype=torch.int32)
        token_indices = torch.arange(input_ids.shape[-1], device=logits.device, dtype=torch.int32)
        last_non_pad_token = (token_indices * non_pad_mask).argmax(-1)

      # Pooling logits from the last non padded token across the batches
      pooled_logits = logits[torch.arange(batch_size, device=logits.device), last_non_pad_token]

      # Calculating loss if labels are provided
      loss = None
      if labels is not None:
        loss = self.loss_function(logits=logits, labels=labels, pooled_logits=pooled_logits, config=self.config)

      return SequenceClassifierOutput(loss=loss, logits=logits)

## Loading tokenizer and model

In [15]:
#tokenizer, data_collator, train_set, eval_set = set_tokenizer(checkpoint)
#base_model = set_model(checkpoint)

In [16]:
#model = DeepseekV2ForSequenceClassification(base_model, base_model.config)

# LoRA

#### Check module names in the model to specify them in *target_modules* param

model = set_model(checkpoint)
for name, module in model.named_modules():
    print(name)

## LoRA config

In [17]:
config = LoraConfig(
    #r=16,
    #lora_alpha=32,
    target_modules = ["q_proj", "k_proj", 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], # Not sure about this
    lora_dropout=0.1,
    bias='none',
    #modules_to_save=['classifier'], # Not sure about this one either
    task_type = "SEQ_CLS"
)

#model = get_peft_model(model=model, peft_config=config)
#model.print_trainable_parameters()

### Flash the drive

!rm -rf training_results

# Trainer Args

In [18]:
def set_training_args(checkpoint, batch_size=16):
    training_args = TrainingArguments(output_dir=f"training_results/{checkpoint}/",
                                      eval_strategy="epoch",
                                      save_strategy="epoch",
                                      logging_strategy="epoch",
                                      #learning_rate=2e-4, # Testing
                                      bf16=True,
                                      report_to='tensorboard',
                                      num_train_epochs=3,
                                      warmup_steps=100, # Testing
                                      per_device_train_batch_size=batch_size,
                                      per_device_eval_batch_size=batch_size,
                                      gradient_accumulation_steps = 4,
                                      # Testing
                                      load_best_model_at_end=True,
                                     )
    return training_args

# Trainer

In [19]:
from accelerate import Accelerator

# Initialize Accelerator
accelerator = Accelerator()

# Set up tokenizer, datasets, and model (as before)
tokenizer, data_collator, train_set, eval_set = set_tokenizer(checkpoint)
base_model = set_model(checkpoint)
model = DeepseekV2ForSequenceClassification(base_model, base_model.config)
model = get_peft_model(model=model, peft_config=config)

# Prepare everything with Accelerator
model, train_set, eval_set, data_collator = accelerator.prepare(
    model, train_set, eval_set, data_collator
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
def finetune(checkpoint):
    # Collecting
    training_args = set_training_args(checkpoint=checkpoint, batch_size=1)

    # Building
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=eval_set,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train
    trainer.train()

    # Save metrics
    test_metrics = trainer.evaluate(eval_dataset=eval_set)
    trainer.save_metrics(split="test", metrics=test_metrics)

    return trainer

trainer = finetune(checkpoint)

/tmp/ipykernel_6509/2096316419.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/venv/main/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 15.60 GiB of which 5.25 MiB is free. Process 3120820 has 15.58 GiB memory in use. Of the allocated memory 14.00 GiB is allocated by PyTorch, and 1.18 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Flushing CUDA

In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()

free_gpu_cache()

# Inference

In [ ]:
device = torch.cuda.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer, data_collator, train_set, eval_set = set_tokenizer(checkpoint)


In [ ]:
def predict(inputs):
    # Tokenizing inputs
    test_sample = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True)
    inputs = Dataset.from_dict({key: value.to(model.device) for key, value in test_sample.items()})

    # Predicting & decoding inputs
    preds = trainer.predict(test_dataset=inputs)
    preds = labelEncoder.inverse_transform(y=np.ravel(np.argmax(preds.predictions[0], axis=-1)))

    return preds

In [ ]:
test_sample = """
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        count = {}
        for num in nums:
            count[num] = 1 + count.get(num, 0)

        arr = []
        for num, cnt in count.items():
            arr.append([cnt, num])
        arr.sort()

        res = []
        while len(res) < k:
            res.append(arr.pop()[1])
        return res
        """

predict(test_sample)